###A Decorator is a callable that takes a callable as input and returns another callable
Decorators allow you to decorate or wrap a function and lets you execute code before or after the wrapped function runs

You can define reusable blocks without permanently modifying the the wrapped function itself
 - Attaches additional responsibilities to an object dynamically
 - Decorator allows you to perform some processing on the decorated function and replace it with another callable.
 - A decorator usually replaces a function with a different one. Its usually accepts the same arguments taken by the decorated function and returns returns whatever the decorated function was supposed to return , while also doing some additional processing




In [0]:
from datetime import datetime
def null_decorator(func):
  """
  A callable that takes another callable as input and returns it without modiying it.
  """
  def new_func():
    print(datetime.now().time())
    val = func()
    print(val)
    print(datetime.now().time())
  return new_func

@null_decorator
def greet():
  return 'Hello!'

@null_decorator
def greet_1():
  return 'Hello_!'

print('Using Decorator')
greet_1()

print('\nAssigning return of decorator to the callable')
# The above code does this:
greet2 = null_decorator(greet)
greet()



Using Decorator
11:48:30.828054
Hello_!
11:48:30.828247

Assigning return of decorator to the callable
11:48:30.828448
Hello!
11:48:30.828545


**greet = null_decorator(greet)** is the same as applying **null_decorator** on the greet function

##When Python executes Decorators

They run right after the decorated function is defined (usually when the module is imported).
The decorator holds references to the functions when imported. They are executed only when explicitly called

 - Decorator 's are executed as soon as the module containing the decorated function is imported. 
 - The decorated function only runs when explicitly evoked

##Closures
- a function that retains the bindings of the free variables that exist when the function is defined.
- these free variables can later be used when the function is invoked and the defining scope is no longer available.
- can happen only in case of nested functions.

In [0]:
  def make_averager():
    series = []
    
    def averager(new_value):
      series.append(new_value)
      total = sum(series)
      return total/len(series)
    return averager
    
avg = make_averager()
avg(10)
      

10.0

make_averager() has the local variable series. make_averager returns averager object.
avg now becomes averager.

On calling avg(10), if now calls averager(). It's local scope is gone and should not know of variable series. 

But, within averager, series is a **free variable**. A ree variable is a variable that is not bound to the local scope. Python keeps these variables in **dunder code**

Another thing to note here is lst.append does not assign a value to the free variable, hence thr variable remains global. This works since lists are mutable. With immutable types like numbers, strings, tuples, you can only read and not update before assigning a new value. 

In [0]:
print(avg.__code__.co_names)
print(avg.__code__.co_freevars)

('append', 'sum', 'len')
('series',)


series is a free variable 

##nonlocal - store the immutable object an inner function needs to change

Another way to create averager would be to store the count of variables, append to the previous sum and calculate average.
In the next example, we are assigning value to an immutable onbject (number). If we use count + 1, we are creating a local variable. It's no longer a free variable and hence not accesible to closure. Hence assignment will fail. Even on using global count, value of count is no longer present since count is no more a free variable and hence not a global variable. 


In [0]:
def make_averager():
  count = 0
  total = 0
  
  def averager(new_value):
    global count, total
    count +=1
    total += new_value
    return total/count
  return averager
 
avrg = make_averager()
avrg(10)

NameError: ignored

To solve this we use nonlocal.
nonlocal lets u flag a variable as free variable even if it's assigned a new value in the closure
If a binding is assigned to a nonlocal variable, the binding stored in the closure is changed.

In [0]:
def make_averager():
  count = 0
  total = 0
  
  def averager(new_value):
    nonlocal count, total
    count +=1
    total += new_value
    return total/count
  return averager
 
avrg = make_averager()
avrg(10)
avrg(11)

10.5

## Simple Decorator

#Multiple Decorators

In [0]:
def add_x(func):
  def wrapper():
    return 'x' + func() + 'x'
  return wrapper

def add_y(func):
  def wrapper():
    return 'y' + func() + 'y'
  return wrapper

@add_x
@add_y
def greet():
  return 'abc'

print(greet())

# This is the same as calling greet = add_x(add_y(greet))
greet1 = add_x(add_y(greet))()
print(greet1)


xyabcyx
xyxyabcyxyx


#Decorating funcions that Accept Arguments

If the decorator is applied to a function with arguments, we can use args and kwargs feature  

Multiple decorators are executed **bottom to top**.  The topmost decorator will be applied last.

In [0]:
def trace(func):
  def wrapper(*args, **kwargs):
    print(f'TRACE: calling function {func.__name__} with arguments {args}, {kwargs}')
    result = func(*args , **kwargs)
    print(result)
  return wrapper

@trace
def name(f_name, l_name):
  return f'{f_name} {l_name}'

name('Gajal', 'Agarwala')

TRACE: calling function name with arguments ('Gajal', 'Agarwala'), {}
Gajal Agarwala
